## Python code

### Imports

In [3]:
import os
import sys
import time
import serial
import math
import numpy as np

# Adding the src folder in the current directory as it contains the script for Thymio and local occupancy
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from Thymio import Thymio

In [4]:
robot = Thymio.serial(port='COM3', refreshing_rate=0.1)

### Accessible functions for the robot

In [5]:
#dir(robot)

### Read/Write variables

In [6]:
#variables = robot.variable_description()
#for var in variables :
#    print(var)

### Variables globales

In [7]:
PROXIMITY_THRESHOLD = 3800 #around 2cm
TIME_SLEEP_OBSTACLE = 0.1 #in seconds
TIME_SLEEP_DATA = 0.5 # 0.2 = 5Hz frequency
MOTOR_SPEED = 80 #both motor are set with this value (one in each direction)

SENSOR_0 = 0
SENSOR_1 = 1
SENSOR_2 = 2
SENSOR_3 = 3
SENSOR_4 = 4
SENSOR_5 = 5
SENSOR_6 = 6
ALL_FRONT_SENSOR = 5
PARTIALY_ALL_FRONT_SENSOR = 4
COMPTEUR = 6
DIST_WHEELS = 9.4

### Main functions

If/elif

Initialisations

In [8]:
def set_course(target, x, y, alpha):
    
    dy = target[1] - y
    dx = target[0] - x
    alphatarg = np.arctan2(dy,dx)
    dalpha = alphatarg - alpha
    
    print('delta ',dx,dy,dalpha)
    
    if  dalpha > 0.05 :
        robot.set_var("motor.left.target", int(2**16)-int(MOTOR_SPEED*dalpha))
        robot.set_var("motor.right.target", int(MOTOR_SPEED*dalpha)) 
    
    elif dalpha < -0.05:
        robot.set_var("motor.left.target", int(MOTOR_SPEED*(-dalpha)))
        robot.set_var("motor.right.target", int(2**16)-int(MOTOR_SPEED*(-dalpha)))
    
    else:
        robot.set_var("motor.left.target", MOTOR_SPEED)
        robot.set_var("motor.right.target", MOTOR_SPEED) 
    return

In [14]:
def obstacle_avoidance(obstacle_direction,sensor_measurment):
    if(np.shape(obstacle_direction)[0] == ALL_FRONT_SENSOR)or(np.shape(obstacle_direction)[0] == PARTIALY_ALL_FRONT_SENSOR):
        if(sensor_measurment[SENSOR_1]>sensor_measurment[SENSOR_3]):
            robot.set_var("motor.left.target", MOTOR_SPEED)
            robot.set_var("motor.right.target", 2**16-MOTOR_SPEED)
        else:
            robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
            robot.set_var("motor.right.target", MOTOR_SPEED)
    else:
        if (np.any(np.equal(obstacle_direction,SENSOR_0)))or(np.any(np.equal(obstacle_direction,SENSOR_1))):
            robot.set_var("motor.left.target", MOTOR_SPEED)
            robot.set_var("motor.right.target", 2**16-MOTOR_SPEED)
        elif (np.any(np.equal(obstacle_direction,SENSOR_3)))or(np.any(np.equal(obstacle_direction,SENSOR_4))):
            robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
            robot.set_var("motor.right.target", MOTOR_SPEED)
        elif (np.any(np.equal(obstacle_direction,SENSOR_2))):
            robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
            robot.set_var("motor.right.target", MOTOR_SPEED)
    return

In [15]:
def go_to_goal(target_list):
    t0 = time.time()
    t1 = time.time()
    x = y = alpha = 0
    target_reached =0
    last_target_reached =0
    follow_obstacle = 0
    iteration = 0
    id_target = 1
    
    while (not last_target_reached) : #simulation of big while loop
        target = target_list[id_target]
        goal = target_list[len(target_list)-1]
        t0 = time.time()
        
        print('target = ', target)

        if follow_obstacle == 1 :
            robot.set_var("motor.left.target", MOTOR_SPEED)        
            robot.set_var("motor.right.target", MOTOR_SPEED)       
        else :
            set_course(target, x, y, alpha)

        sensor_measurment = robot["prox.horizontal"]

        if np.all(np.less(sensor_measurment, PROXIMITY_THRESHOLD)):
            pass
        else:
            obstacle_direction = np.nonzero(sensor_measurment)[0]
            while((np.any(np.greater(sensor_measurment, PROXIMITY_THRESHOLD)))and(np.all(np.not_equal(obstacle_direction, SENSOR_5)))and(np.all(np.not_equal(obstacle_direction, SENSOR_6)))):
                #robot.set_var('leds.top', 123)

                timer = t1-t0
                t0 = time.time()

                lspeed = robot.get_var("motor.left.target")
                rspeed = robot.get_var("motor.right.target")

                if (lspeed > 100):
                    lspeed = lspeed - int(2**16)
                    d = (-lspeed+rspeed)*timer/45
                    alpha = alpha + np.arcsin(2*d/DIST_WHEELS)

                elif (rspeed > 100):
                    rspeed = rspeed - int(2**16)
                    d = (lspeed-rspeed)*timer/45
                    alpha = alpha - np.arcsin(2*d/DIST_WHEELS)
                else:
                    d = (lspeed+rspeed)*timer/45
                    x=x+d*np.cos(alpha)
                    y=y+d*np.sin(alpha)

                print('paramètres ',timer, lspeed, rspeed, d)

                print('robot = ',x,y,alpha)

                obstacle_avoidance(obstacle_direction,sensor_measurment)

                time.sleep(TIME_SLEEP_OBSTACLE)

                sensor_measurment = robot["prox.horizontal"]
                obstacle_direction = np.nonzero(sensor_measurment)[0]
                t1 = time.time()
                
            id_target = id_target + 1
            if id_target>len(target_list)-1:
                id_target = len(target_list)-1
            
        if (sensor_measurment[0] > 1000) or (sensor_measurment[4] > 1000) :
            follow_obstacle = 1
            iteration = 0
        else : 
            iteration = iteration + 1
            if iteration == COMPTEUR :
                follow_obstacle = 0
                

        time.sleep(TIME_SLEEP_DATA)
        t1 = time.time()
        timer = t1-t0
        

        lspeed = robot.get_var("motor.left.target")
        rspeed = robot.get_var("motor.right.target")

        if (lspeed > 100):
            lspeed = lspeed - int(2**16)
            d = (-lspeed+rspeed)*timer/45
            alpha = alpha + np.arcsin(2*d/DIST_WHEELS)

        elif (rspeed > 100):
            rspeed = rspeed - int(2**16)
            d = (lspeed-rspeed)*timer/45
            alpha = alpha - np.arcsin(2*d/DIST_WHEELS)
        else:
            d = (lspeed+rspeed)*timer/45
            x=x+d*np.cos(alpha)
            y=y+d*np.sin(alpha)


        print('paramètres ',timer, lspeed, rspeed, d)

        print('robot = ',x,y,alpha)

        if -0.75 < x-target[0] < 0.75 and -0.75 < y-target[1] < 0.75 :
            target_reached = 1
            print('coucou')
            id_target = id_target+1
            if id_target>len(target_list)-1:
                id_target = len(target_list)-1
            
        if -1 < x-goal[0] < 1 and -1 < y-goal[1] < 1 :
            last_target_reached = 1
            robot.set_var("motor.left.target", 0)
            robot.set_var("motor.right.target", 0)

    return

In [16]:
#Main

target_list = [[0,0],[10,10],[20,20],[30,30],[40,40]]
go_to_goal(target_list)

target =  [10, 10]
delta  10 10 0.7853981633974483
paramètres  0.5019981861114502 -62 62 1.3832838906182183
robot =  0 0 0.2987394485644006
target =  [10, 10]
delta  10 10 0.4866587148330477
paramètres  0.5047838687896729 -38 38 0.8525238672892252
robot =  0 0 0.4811371875098892
target =  [10, 10]
delta  10 10 0.30426097588755907
paramètres  0.507692813873291 -24 24 0.5415390014648438
robot =  0 0 0.5966147306348958
target =  [10, 10]
delta  10 10 0.1887834327625525
paramètres  0.5041708946228027 -15 15 0.33611392974853516
robot =  0 0 0.6681894291748054
target =  [10, 10]
delta  10 10 0.11720873422264289
paramètres  0.5042524337768555 -9 9 0.20170097351074218
robot =  0 0 0.7111177136831467
target =  [10, 10]
delta  10 10 0.07428044971430159
paramètres  0.5053751468658447 -5 5 0.11230558819240993
robot =  0 0 0.7350147941026764
target =  [10, 10]
delta  10 10 0.05038336929477183
paramètres  0.5035398006439209 -4 4 0.0895181867811415
robot =  0 0 0.7540623685765534
target =  [10, 10]
d

<ipython-input-15-b70c4b34a5bf>:96: RuntimeWarning: invalid value encountered in arcsin
  alpha = alpha + np.arcsin(2*d/DIST_WHEELS)


paramètres  0.5057532787322998 -65387 65387 1469.7639838430616
robot =  -0.6628633463656055 13.933850797328144 nan
target =  [20, 20]
delta  20.662863346365604 6.0661492026718555 nan
paramètres  0.5037133693695068 80 80 1.7909808688693576
robot =  nan nan nan
target =  [20, 20]
delta  nan nan nan
paramètres  0.5036458969116211 80 80 1.790740966796875
robot =  nan nan nan
target =  [20, 20]
delta  nan nan nan
paramètres  0.5053949356079102 80 80 1.7969597710503473
robot =  nan nan nan
target =  [20, 20]
delta  nan nan nan
paramètres  0.5051364898681641 80 80 1.7960408528645833
robot =  nan nan nan
target =  [20, 20]
delta  nan nan nan
paramètres  0.5066640377044678 80 80 1.80147213406033
robot =  nan nan nan
target =  [20, 20]
delta  nan nan nan
paramètres  0.5047042369842529 80 80 1.7945039537217882
robot =  nan nan nan
target =  [20, 20]
delta  nan nan nan
paramètres  0.5083441734313965 80 80 1.8074459499782987
robot =  nan nan nan
target =  [20, 20]
delta  nan nan nan
paramètres  0.5

paramètres  0.10545730590820312 -80 80 0.3749593098958333
robot =  nan nan nan
paramètres  0.10389947891235352 -80 80 0.3694203694661458
robot =  nan nan nan
paramètres  0.10548186302185059 -80 80 0.37504662407769096
robot =  nan nan nan
paramètres  0.1081390380859375 -80 80 0.3844943576388889
robot =  nan nan nan
paramètres  0.11028909683227539 80 -80 0.3921390109592014
robot =  nan nan nan
paramètres  0.11554574966430664 -80 80 0.4108293321397569
robot =  nan nan nan
paramètres  0.10745096206665039 -80 80 0.3820478651258681
robot =  nan nan nan
paramètres  0.10853743553161621 -80 80 0.38591088189019096
robot =  nan nan nan
paramètres  0.10744357109069824 -80 80 0.38202158610026044
robot =  nan nan nan
paramètres  0.11050963401794434 -80 80 0.39292314317491317
robot =  nan nan nan
paramètres  0.10849595069885254 80 -80 0.38576338026258683
robot =  nan nan nan
paramètres  0.10780930519104004 80 -80 0.38332197401258683
robot =  nan nan nan
paramètres  0.14161920547485352 80 -80 0.503534

paramètres  0.5036137104034424 80 80 1.7906265258789062
robot =  nan nan nan
target =  [40, 40]
paramètres  -0.006975889205932617 80 80 -0.02480316162109375
robot =  nan nan nan
paramètres  0.11436152458190918 80 80 0.40661875406901044
robot =  nan nan nan
paramètres  0.10607075691223145 80 -80 0.37714046902126735
robot =  nan nan nan
paramètres  0.10563540458679199 80 -80 0.37559254964192706
robot =  nan nan nan
paramètres  0.6065225601196289 80 -80 2.156524658203125
robot =  nan nan nan
target =  [40, 40]
paramètres  0.5033130645751953 80 80 1.7895575629340277
robot =  nan nan nan
target =  [40, 40]
paramètres  0.5031778812408447 80 80 1.789076911078559
robot =  nan nan nan
target =  [40, 40]
paramètres  0.5054662227630615 80 80 1.7972132364908855
robot =  nan nan nan
target =  [40, 40]
paramètres  0.5043661594390869 80 80 1.7933019002278645
robot =  nan nan nan
target =  [40, 40]
paramètres  0.5056514739990234 80 80 1.7978719075520833
robot =  nan nan nan
target =  [40, 40]
paramètr

KeyboardInterrupt: 

### Tests

ANN

In [ ]:
WEIGHT_L=[40,25,-20,-25,-40, 0,0] #sensor_5 = 30 ; sensor_6 = -10 (here = 0 beacause of the cable)
WEIGHT_R=[-40,-25,-20,25,40, 0,0] #sensor_5 = -10 ; sensor_6 = 30 (here = 0 beacause of the cable)

SENSOR_SCALE = 4000 #used for rescaling the data send by the sensor to give it to the motors
THRESHOLD = 20
MOTOR_SPEED = 80 #both motor are set with this value (one in each direction)

for i in range(10000000): #simulation of big while loop 
    
    robot.set_var("motor.left.target", int(2**16) - MOTOR_SPEED)        #TO REMOVE
    robot.set_var("motor.right.target", MOTOR_SPEED)       #TO REMOVE
    
    sensor_measurment = robot["prox.horizontal"]
    #print(sensor_measurment)
    
    measurement_rescaled = np.divide(sensor_measurment, SENSOR_SCALE)
    #print(measurement_rescaled)
    weighted_measurment_l = np.multiply(measurement_rescaled, WEIGHT_L)
    #print(weighted_measurment_l)
    weighted_measurment_r = np.multiply(measurement_rescaled, WEIGHT_R)
    #print(weighted_measurment_r)

    motor_speed_l = np.sum(weighted_measurment_l)
    motor_speed_r = np.sum(weighted_measurment_r)

    #print(motor_speed_l)
    #print(motor_speed_r)
    
    if np.any(np.greater(motor_speed_l, THRESHOLD)) or np.any(np.greater(motor_speed_r, THRESHOLD)) or np.any(np.less(motor_speed_l, -THRESHOLD)) or np.any(np.less(motor_speed_r, -THRESHOLD)):       
        
        obstacle_direction = np.nonzero(sensor_measurment)[0]
        
        if (np.any(np.equal(obstacle_direction,SENSOR_2))) and (np.shape(obstacle_direction)[0]==1):
                    robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                    robot.set_var("motor.right.target", MOTOR_SPEED)
        else :
            if motor_speed_l<0:
                #print(2**16 + int(motor_speed_l))
                robot.set_var("motor.left.target", int(2**16) + int(motor_speed_l)-1)
            else :
                robot.set_var("motor.left.target", int(motor_speed_l))

            if motor_speed_r<0:
                #print(2**16 + int(motor_speed_r))
                robot.set_var("motor.right.target", int(2**16) + int(motor_speed_r)-1)
            else :
                robot.set_var("motor.right.target", int(motor_speed_r))
        
        
        """
        robot.set_var("motor.left.target", 0)
        robot.set_var("motor.right.target", 0)
        
        robot.set_var("motor.left.target", 2**16-motor_speed_l)
        robot.set_var("motor.right.target", 2**16-motor_speed_r)
        """
    
    time.sleep(TIME_SLEEP_OBSTACLE)

    

In [17]:
sensor_measurment = robot["prox.horizontal"]
obstacle_direction = np.nonzero(sensor_measurment)[0]

print(sensor_measurment)
print(obstacle_direction)
#print(np.argmax(sensor_measurment))
print((np.shape(obstacle_direction)[0] == 1) and (np.any(np.equal(obstacle_direction,SENSOR_0)) or np.any(np.equal(obstacle_direction,SENSOR_4))))
print(np.any(np.greater(sensor_measurment, PROXIMITY_THRESHOLD)))
print(np.all(np.not_equal(obstacle_direction, SENSOR_5)))
print(np.all(np.not_equal(obstacle_direction, SENSOR_6)))
print(np.all(np.less(sensor_measurment, PROXIMITY_THRESHOLD)))
print(np.any(np.equal(obstacle_direction,SENSOR_2)))

#robot.set_var("motor.left.target", 50)
#robot.set_var("motor.right.target", 2**16-50)
#time.sleep(1.1)
robot.set_var("motor.left.target", 0)
robot.set_var("motor.right.target", 0)
              


[0, 0, 0, 0, 0, 0, 0]
[]
False
False
True
True
True
False


Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\roxan\Documents\Applications\Anaconda\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\roxan\Documents\Applications\Anaconda\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\roxan\Documents\EPFL_COURS\Basics of mobile robotics\Solutions_4\Solutions\src\Thymio.py", line 340, in do_refresh
    self.get_variables()
  File "C:\Users\roxan\Documents\EPFL_COURS\Basics of mobile robotics\Solutions_4\Solutions\src\Thymio.py", line 499, in get_variables
    self.send(msg)
  File "C:\Users\roxan\Documents\EPFL_COURS\Basics of mobile robotics\Solutions_4\Solutions\src\Thymio.py", line 456, in send
    self.io.write(msg.serialize())
  File "C:\Users\roxan\Documents\Applications\Anaconda\lib\site-packages\serial\serialwin32.py", line 323, in write
    raise writeTimeoutError
serial.serialutil.SerialTimeoutException: Write timeout


### Bugs to fix

1) if the robot meet a v shape obstacle  
2) if the robot has an obstacle in front of sensor n°2 it can turns and then sense the obstacle on the 3th, it turns backwards --> wait a bit more in the case of detection on the sensor 2 ?